In [1]:
import requests

In [2]:
response = requests.get("http://newsapi.org/v2/top-headlines?country=in&category=entertainment&apiKey=469e3b5dc7b44bf6ae90867fe0e8fd5c")

In [3]:
data = response.json()

In [4]:
from types import SimpleNamespace
import json

In [5]:
articles = data['articles']

In [6]:
news = []
for article in articles:
    title_parts = article['title'].split(" - ")
    text = title_parts[0] + article['description']
    news.append(text)

In [7]:
import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())
import pickle

tokenizer_path = Path('../datasets/sentiment_analysis/tokenizer.pickle').resolve()
with tokenizer_path.open('rb') as file:
    tokenizer = pickle.load(file)

from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Conv1D, Dense, concatenate
from tensorflow.keras.models import Model

input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
num_classes = 4
embedding_dim = 500
input_length = 100
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

input_layer = Input(shape=(input_length,))
output_layer = Embedding(
  input_dim=input_dim,
  output_dim=embedding_dim,
  input_shape=(input_length,)
)(input_layer)

output_layer = SpatialDropout1D(spatial_dropout)(output_layer)

output_layer = Bidirectional(
LSTM(lstm_units, return_sequences=True,
     dropout=lstm_dropout, recurrent_dropout=recurrent_dropout)
)(output_layer)
output_layer = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                    kernel_initializer='glorot_uniform')(output_layer)

avg_pool = GlobalAveragePooling1D()(output_layer)
max_pool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avg_pool, max_pool])

output_layer = Dense(num_classes, activation='softmax')(output_layer)

model = Model(input_layer, output_layer)

model_weights_path = Path('../models/emotion_recognition/model_weights.h5').resolve()
model.load_weights(model_weights_path.as_posix())

import pandas as pd

encoder_path = Path('../models/emotion_recognition/encoder.pickle').resolve()
with encoder_path.open('rb') as file:
    encoder = pickle.load(file)
    
from nlp import preprocess
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
subs = "There was an idea, to bring together a group of remarkable people, to see if we could become something more. So when they needed us we could fight the battles that they never could. In time, you'll know what it's like to lose. To feel so desperately that you're right, that to fail all the same. Dread it, Run from it. Destiny still arrives. Evacuate the city. Engage all defences and get this man a shield. Fun isn't something one considers the balance in the universe. But this does put a smile on my face. Who the hell are you guys?"
dialogs = subs.split(". ")

In [9]:
selected = dialogs[9]
sequences = [text.split() for text in selected]
list_tokenized = tokenizer.texts_to_sequences(sequences)
x_data = pad_sequences(list_tokenized, maxlen=100)

y_pred = model.predict(x_data)
emo = dict()
for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
    print(encoder.classes_[index] + ": " + str(value))
    emo[encoder.classes_[index]]=value

anger: 0.13067813
fear: 0.7585651
joy: 0.021767307
sadness: 0.08898952


In [10]:
min_diff = 5
ans = 0
for i in range(len(news)):
    cleaned_data = news[i]
    sequences = [text.split() for text in cleaned_data]
    list_tokenized = tokenizer.texts_to_sequences(sequences)
    x_data = pad_sequences(list_tokenized, maxlen=100)

    y_pred = model.predict(x_data)
    
    cur_diff = 0
    for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
        cur_diff += abs(emo[encoder.classes_[index]] - value)

    if cur_diff < min_diff:
        ans = i
        min_diff = cur_diff


In [11]:
print(news[ans] + " : " + selected)

Amitabh Bachchan hangs 'nimbu mirchi' on 2021 to protect it from evil eyeMegastar Amitabh Bachchan on Tuesday shared a quirky post in an attempt to protect the year 2021 from the evil eye. : But this does put a smile on my face


In [12]:
cleaned_data = news[1]
sequences = [text.split() for text in cleaned_data]
list_tokenized = tokenizer.texts_to_sequences(sequences)
x_data = pad_sequences(list_tokenized, maxlen=100)

y_pred = model.predict(x_data)
for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
    print(encoder.classes_[index] + ": " + str(value))

anger: 0.13310911
fear: 0.7552341
joy: 0.021626506
sadness: 0.090030156


In [23]:
def getMeme(dialogs,news):
    movie_line = 0
    pairs = dict()
    global_diff = 6
    for j in range(len(dialogs)):
        selected = dialogs[j]
        sequences = [text.split() for text in selected]
        list_tokenized = tokenizer.texts_to_sequences(sequences)
        x_data = pad_sequences(list_tokenized, maxlen=100)

        y_pred = model.predict(x_data)
        emo = dict()
        for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
            print(encoder.classes_[index] + ": " + str(value))
            emo[encoder.classes_[index]]=value

        min_diff = 3
        news_line = 0
        for i in range(len(news)):
            cleaned_data = news[i]
            sequences = [text.split() for text in cleaned_data]
            list_tokenized = tokenizer.texts_to_sequences(sequences)
            x_data = pad_sequences(list_tokenized, maxlen=100)

            y_pred = model.predict(x_data)

            cur_diff = 0
            for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
                cur_diff += abs(emo[encoder.classes_[index]] - value)

            if cur_diff < min_diff:
                news_line = i
                min_diff = cur_diff

        if min_diff < global_diff:
            movie_line = j
            global_diff = min_diff
            
        print(articles[news_line]['title'] + " : " + dialogs[j] )
        pairs[j] = news_line
    
    print("Final Output --> "+ news[pairs[movie_line]] + " : " + dialogs[movie_line])


In [24]:
getMeme(dialogs,news)

anger: 0.13230877
fear: 0.75031745
joy: 0.020764206
sadness: 0.0966099
Monal Misses Top 5, But Takes Home Rs 50 Lakh!.. - Greatandhra.com : There was an idea, to bring together a group of remarkable people, to see if we could become something more
anger: 0.13240752
fear: 0.75724274
joy: 0.022118635
sadness: 0.08823136
Amitabh Bachchan hangs 'nimbu mirchi' on 2021 to protect it from evil eye - Times of India : So when they needed us we could fight the battles that they never could
anger: 0.13177775
fear: 0.7610933
joy: 0.022968303
sadness: 0.0841608
Amitabh Bachchan hangs 'nimbu mirchi' on 2021 to protect it from evil eye - Times of India : In time, you'll know what it's like to lose
anger: 0.13157234
fear: 0.7553917
joy: 0.021886934
sadness: 0.09114935
Photos of celebrities at airport Pics | Photos of celebrities at airport Photos | Photos of celebrities at airport Portfolio Pics | Photos of celebrities at airport Personal Photos - ETimes Photogallery - Times of India : To feel so desp